# Homework 1

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 13, 2020)_

This Notebook will walk you through defining a simple transport flow model and then ask you to interact with the solutions and modify to model to add additional constraints...

## Setting up the model

### Load packages

In [1]:
using JuMP
using Clp
using DataFrames
using CSV

### Define sets

We will define two sets, both as arrays of strings

***Production plants, $P$***

In [2]:
P=["trenton", "newark"] # production plants

2-element Array{String,1}:
 "trenton"
 "newark"

***Markets for products, $M$***

In [3]:
M=["newyork", "princeton", "philadelphia"] # markets for products

3-element Array{String,1}:
 "newyork"
 "princeton"
 "philadelphia"

Note that sets can also be defined over intervals (as in `i=1:10`) or numerical vectors (as in `x=[2, 4, 5, 11]`) 

### Define parameters

We'll make use of the defined sets as indexes for our parameters...

***Plant production capacities***

In [4]:
plants = DataFrame(plant=P, capacity=[350,650])

2×2 DataFrame
│ Row │ plant   │ capacity │
│     │ String  │ Int64    │
├─────┼─────────┼──────────┤
│ 1   │ trenton │ 350      │
│ 2   │ newark  │ 650      │

***Demand for products***

Stored in a [DataFrame](https://juliadata.github.io/DataFrames.jl/stable/)

In [5]:
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275]
)

3×2 DataFrame
│ Row │ market       │ demand │
│     │ String       │ Int64  │
├─────┼──────────────┼────────┤
│ 1   │ newyork      │ 325    │
│ 2   │ princeton    │ 300    │
│ 3   │ philadelphia │ 275    │

A few different ways to index into our DataFrames to access parameters (all of the below are equivalent)

In [6]:
plants[plants.plant.=="newark",:capacity] # option 1

1-element Array{Int64,1}:
 650

In [7]:
plants[plants.plant.=="newark",:].capacity # option 2

1-element Array{Int64,1}:
 650

In [8]:
plants.capacity[plants.plant.=="newark"] # option 3

1-element Array{Int64,1}:
 650

In [9]:
plants[:,:capacity][plants.plant.=="newark"] # option 4

1-element Array{Int64,1}:
 650

Note that DataFrame indexing returns an Array by default, in this case, a 1-element Array of type Int64 (64-bit integer), as indicated by `Array{Int64,1}` above. 

To access the single Int64 value, append `[1]` to any of the above to reference the first (and only) element in this array. 

In [10]:
plants.capacity[plants.plant.=="newark"][1]

650

In [11]:
typeof(plants.capacity[plants.plant.=="newark"][1])

Int64

In [12]:
typeof(plants.capacity[plants.plant.=="newark"])

Array{Int64,1}

***Distance from plants to markets***

Stored in a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/v0.19/containers/) with data array and symbolic references across each of our sets (plants and markets), converted to Symbols for referencing

In [13]:
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5;
     0.5 1.5 3.5],
    Symbol.(P),
    Symbol.(M),
)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia]
And data, a 2×3 Array{Float64,2}:
 2.5  0.5  1.5
 0.5  1.5  3.5

A couple example references to our DenseAxisArray to access parameters...

In [14]:
distances[:trenton, :newyork] #example of distance references

2.5

In [15]:
distances[:newark, :newyork] #example of distance references

0.5

In [16]:
distances[Symbol(P[2]),Symbol(M[1])] # another way to find distance from newark to trenton

0.5

In [17]:
distances[Symbol("newark"), Symbol("newyork")] # and a third...

0.5

***Costs of transport***

In [18]:
freight_cost = 90 # Cost of freight shipment per unit of distance

90

### Create model
(and specify the Clp solver)

In [28]:
transport = Model(Clp.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp

### Define variables

***Quantities of product to transport from plant $p \in P$ to market $m \in M$***

In [29]:
@variable(transport, X[P,M] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Array{VariableRef,2}:
 X[trenton,newyork]  X[trenton,princeton]  X[trenton,philadelphia]
 X[newark,newyork]   X[newark,princeton]   X[newark,philadelphia]

Example reference to single quantity decision variable, the quantity shipped from Newark to Philadelphia:

In [30]:
X["newark","philadelphia"]

X[newark,philadelphia]

### Define constraints

***Supply capacity constraint***

In [31]:
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, ["trenton", "newark"]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 cSupply[trenton] : X[trenton,newyork] + X[trenton,princeton] + X[trenton,philadelphia] <= 350.0
 cSupply[newark] : X[newark,newyork] + X[newark,princeton] + X[newark,philadelphia] <= 650.0

***Demand balance constraint***

Ensure all demand is satisfied at each market

In [32]:
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, ["newyork", "princeton", "philadelphia"]
And data, a 3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1}:
 cDemand[newyork] : X[trenton,newyork] + X[newark,newyork] >= 325.0
 cDemand[princeton] : X[trenton,princeton] + X[newark,princeton] >= 300.0
 cDemand[philadelphia] : X[trenton,philadelphia] + X[newark,philadelphia] >= 275.0

### Define objective function

Minimize total cost of transport to satisfy all demand.

First we'll define an expression for total cost of shipments...

In [33]:
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

Now we'll minimize this total cost

In [34]:
@objective(transport, Min, eCost)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

## Interact with the model

**(a)** Now let's solve the model. In the blank cell below, enter the command for JuMP to solve a model and run the cell

In [36]:
optimize!(transport)

Coin0506I Presolve 5 (0) rows, 6 (0) columns and 12 (0) elements
Clp0006I 0  Obj 0 Primal inf 900 (3)
Clp0006I 4  Obj 85500
Clp0000I Optimal - objective value 85500
Clp0032I Optimal objective 85500 - 4 iterations time 0.002


**(b)** You've got a solution. Now query the objective function in the empty cell below and save it to a variable (name of your choice)

In [37]:
f

85500.0

**(c)** Now query and save the optimal solution for X (the decisions about shipment quantities from plant to market) to an Array or DataFrame

In [65]:
solutionX = value.(X)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Array{Float64,2}:
   0.0   75.0  275.0
 325.0  225.0    0.0

**(d)** Please interpret your results by writing an explanation in the markdown cell below. 

Which facility or facilities supplies the most demand in New York? Does this result make sense? Why?

Which facility or facilities supplies the most demand in Philadelphia? Does this result make sense? Why?

Which facility or facilities supplies the demand in Princeton? Does this result make sense? Why?



Facilty for New York: Newark. Yes, it makes sense because Newark is the closest plant to New York.
Facilty for Philadelphia: Newark. Yes, it makes sense because it is most cost effcient to use Trenton to supply Princeton first, then Trenton's remaining capacity was used in Philadelphia (75). Then, Newark had to satisfy the remaining demand (225).
Facilty for Princeton: Trenton. Yes, it makes sense because Trenton is the closest plant to New York.

**(d)** A new market in New Brunswick appears, with a demand for 50 units. It is located 1.0 units away from both plants. Add this market to the model and solve again.

In [77]:
M=["newyork", "princeton", "philadelphia","newbrunswick"] # markets for products
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275,50]
)
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5 1.0;
     0.5 1.5 3.5 1.0],
    Symbol.(P),
    Symbol.(M),
)
transport2 = Model(Clp.Optimizer)
@variable(transport2, X1[P,M] >= 0)
@constraint(transport2, cSupply[p in P], 
    sum(X1[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])
@constraint(transport2, cDemand[m in M], 
    sum(X1[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])
@expression(transport2, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X1[p,m] 
        for p in P, m in M)
    )
@objective(transport2, Min, eCost)
optimize!(transport2)

Coin0506I Presolve 6 (0) rows, 8 (0) columns and 16 (0) elements
Clp0006I 0  Obj 0 Primal inf 950 (4)
Clp0006I 5  Obj 90000
Clp0000I Optimal - objective value 90000
Clp0032I Optimal objective 90000 - 5 iterations time 0.002


**(e)** What is new optimal solution? 

In [78]:
solutionX1 = value.(X1)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia", "newbrunswick"]
And data, a 2×4 Array{Float64,2}:
   0.0   75.0  275.0   0.0
 325.0  225.0    0.0  50.0

**(f)** Interpret this result in the markdown cell below. Which facility or facilities supplies the demand in New Brunswick? Does this result make sense? Why?

The facility that supplies New Brunswick's demand is Newark. This make sense because it is the same cost for both plants to get to New Brunswick, and Trenton is already using all of its capacity for Princeton and Philadelphia.